# Neurostock - Team Galaxy - Samsung Innovation Campus 2024 - 2025

## Paso 1: Preparación de los Datos

In [36]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader, TensorDataset
data_folder = r"C:\Users\mleonet_\Desktop\FinalProyect-TeamGalaxy-SIC\Data"
companies = [
    "AAPL", #0
    "ADBE", #1
    "AMZN", #2
    "CSCO", #3
    "DELL", #4
    "GOOGL", #5
    "IBM", #6
    "INTC", #7
    "META", #8
    "MSFT", #9
    "NOK", #10
    "NTDOY", #11
    "NVDA", #12
    "NFLX", #13
    "ORCL", #14
    "QCOM", #15
    "SONY", #16
    "SSNLF", #17
    "TSLA", #18
]

data_paths = {}
# Generación de rutas automatizada para cada Data de cada empresa
for ticker in companies:
    data_paths[ticker] = {
        "historical_data": f"{data_folder}/Historical_Data/{ticker}_historical_data.csv".replace("\\", "/"),
        "balance_sheet": f"{data_folder}/Balance_Sheet_Data/{ticker}_balance_sheet.csv".replace("\\", "/"),
        "cash_flow": f"{data_folder}/Cash_Flow_Data/{ticker}_cash_flow.csv".replace("\\", "/"),
        "income_statement": f"{data_folder}/Income_Statement_Data/{ticker}_income_statement.csv".replace("\\", "/"),
        "financial_ratios": f"{data_folder}/Financial_Ratios_Data/{ticker}_financial_ratios.csv".replace("\\", "/"),
    }

### Funciones para manejar declaraciones de Dataframes

In [37]:
def load_Historical_Data(ticker, data = data_paths):
    historical_csv_path = data_paths[ticker]["historical_data"]
    return pd.read_csv(historical_csv_path)

def load_Balance_Sheet(ticker, data = data_paths):
    Balance_Sheet_csv_path = data_paths[ticker]["balance_sheet"]
    return pd.read_csv(Balance_Sheet_csv_path)

def load_Cash_Flow(ticker, data = data_paths):
    Cash_Flow_csv_path = data_paths[ticker]["cash_flow"]
    return pd.read_csv(Cash_Flow_csv_path)

def load_Income_Statement(ticker, data = data_paths):
    Income_Statement_csv_path = data_paths[ticker]["income_statement"]
    return pd.read_csv(Income_Statement_csv_path)

def load_Financial_Ratios(ticker, data = data_paths):
    Financial_Ratios_csv_path = data_paths[ticker]["financial_ratios"]
    return pd.read_csv(Financial_Ratios_csv_path)

### Limpieza de Dataset

In [38]:
# Dataset de prueba
ticker = companies[17]
historical_data = load_Historical_Data(ticker)

# Verificar que las columnas sean las esperadas
expected_columns = ["Date","Open","High","Low","Close", "Volume","Dividends","Stock Splits","Adj Close"]
actual_columns = historical_data.columns.tolist()

if actual_columns == expected_columns:
    historical_data = historical_data.drop(columns=["Dividends", "Stock Splits"])

# Ajustando el formato de las fechas
historical_data["Date"] = pd.to_datetime(historical_data["Date"], errors="coerce", utc=True)
historical_data["Date"] = historical_data["Date"].dt.tz_localize(None)
historical_data["Date"] = historical_data["Date"].dt.strftime("%Y-%m-%d")

# Limpiar Data
historical_data = historical_data.dropna()

##### Samsung Stocks KRW to USD

In [39]:
if ticker == companies[17]:
    won_to_usd_00_17 = pd.read_csv(
        r"C:\Users\mleonet_\Desktop\FinalProyect-TeamGalaxy-SIC\Data\Won_Conversion_Data\KRW_TO_USD_2000-2017.csv"
    )
    won_to_usd_04_22 = pd.read_csv(
        r"C:\Users\mleonet_\Desktop\FinalProyect-TeamGalaxy-SIC\Data\Won_Conversion_Data\KRW_TO_USD_2004-2022.csv"
    )
    # Preparar Datos de conversión
    won_to_usd_04_22 = won_to_usd_04_22[
        ["Date", "KRW=X"]
    ]  # dejar solamente conversion de wones
    # Borrar filas con datos perdidos
    historical_data = historical_data[historical_data["Volume"] != 0]
    won_to_usd_00_17 = won_to_usd_00_17[won_to_usd_00_17["DEXKOUS"] != "."]
    won_to_usd_04_22 = won_to_usd_04_22[won_to_usd_04_22["KRW=X"] != "."]
    # Renombrar columnas
    won_to_usd_00_17 = won_to_usd_00_17.rename(
        columns={"DEXKOUS": "Value", "DATE": "Date"}
    )
    won_to_usd_04_22 = won_to_usd_04_22.rename(columns={"KRW=X": "Value"})
    # Ajustar formato de fechas
    won_to_usd_00_17["Date"] = pd.to_datetime(won_to_usd_00_17["Date"], errors="coerce")
    won_to_usd_00_17["Date"] = won_to_usd_00_17["Date"].dt.tz_localize(None)
    won_to_usd_00_17["Date"] = won_to_usd_00_17["Date"].dt.strftime("%Y-%m-%d")
    won_to_usd_00_17["Value"] = won_to_usd_00_17["Value"].astype(float, errors="ignore")
    won_to_usd_04_22["Date"] = pd.to_datetime(won_to_usd_04_22["Date"], errors="coerce")
    won_to_usd_04_22["Date"] = won_to_usd_04_22["Date"].dt.tz_localize(None)
    won_to_usd_04_22["Date"] = won_to_usd_04_22["Date"].dt.strftime("%Y-%m-%d")
    won_to_usd_04_22["Value"] = won_to_usd_04_22["Value"].astype(float, errors="ignore")
    # Eliminar filas vacias
    won_to_usd_00_17 = won_to_usd_00_17.dropna()
    won_to_usd_04_22 = won_to_usd_04_22.dropna()
    # Unir los datasets
    won_combined = pd.concat([won_to_usd_00_17, won_to_usd_04_22])
    combined_dollar_values = won_combined.drop_duplicates(subset="Date")
    historical_data = pd.merge(
        historical_data, combined_dollar_values, on="Date", how="left"
    )
    historical_data = historical_data.dropna()
    # Convertir a USD
    historical_data["Close"] = historical_data["Close"] / historical_data["Value"]
    historical_data["Open"] = historical_data["Open"] / historical_data["Value"]
    historical_data["High"] = historical_data["High"] / historical_data["Value"]
    historical_data["Low"] = historical_data["Low"] / historical_data["Value"]
    historical_data["Adj Close"] = (
        historical_data["Adj Close"] / historical_data["Value"]
    )
    # Eliminar columna Value
    historical_data = historical_data.drop(columns=["Value"])

### Funciones para Calculo de Indices Básicos

In [40]:
# Calculo del SMA: Simple Moving Average
historical_data["SMA_5"] = historical_data["Close"].rolling(window=5).mean()
historical_data["SMA_10"] = historical_data["Close"].rolling(window=10).mean()
historical_data["SMA_20"] = historical_data["Close"].rolling(window=20).mean()
historical_data["SMA_50"] = historical_data["Close"].rolling(window=50).mean()
historical_data["SMA_100"] = historical_data["Close"].rolling(window=100).mean()
historical_data["SMA_200"] = historical_data["Close"].rolling(window=200).mean()

# Calculo del RSI: Relative Strength Index
window = 14
delta = historical_data["Close"].diff()
gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
rs = gain / loss
historical_data["RSI_14"] = 100 - (100 / (1 + rs))

# Calculo del MACD: Moving Average Convergence Divergence
short_window = 12
long_window = 26
signal_window = 9
short_ema = historical_data["Close"].ewm(span=short_window, adjust=False).mean()
long_ema = historical_data["Close"].ewm(span=long_window, adjust=False).mean()
historical_data["MACD"] = short_ema - long_ema
historical_data["Signal_Line"] = (
    historical_data["MACD"].ewm(span=signal_window, adjust=False).mean()
)
historical_data = historical_data.dropna()

### New Features

In [41]:
# Calcular las nuevas features usando .loc
historical_data.loc[:, "EMA_50"] = (
    historical_data["Close"].ewm(span=50, adjust=False).mean()
)
historical_data.loc[:, "Daily_Return"] = historical_data["Close"].pct_change()
historical_data.loc[:, "Log_Return"] = np.log(
    historical_data["Close"] / historical_data["Close"].shift(1)
)
historical_data.loc[:, "Middle_Band"] = (
    historical_data["Close"].rolling(window=20).mean()
)
historical_data.loc[:, "Std"] = historical_data["Close"].rolling(window=20).std()
historical_data.loc[:, "Upper_Band"] = (
    historical_data["Middle_Band"] + 2 * historical_data["Std"]
)
historical_data.loc[:, "Lower_Band"] = (
    historical_data["Middle_Band"] - 2 * historical_data["Std"]
)
# Puedes eliminar la columna temporal "Std"
historical_data = historical_data.drop(columns=["Std"])

# Calcular Delta, Gain, Loss y Rs
historical_data.loc[:, "Delta"] = historical_data["Close"].diff()
historical_data.loc[:, "Gain"] = historical_data["Delta"].apply(
    lambda x: x if x > 0 else 0
)
historical_data.loc[:, "Loss"] = historical_data["Delta"].apply(
    lambda x: -x if x < 0 else 0
)
historical_data.loc[:, "Avg_Gain"] = (
    historical_data["Gain"].rolling(window=14, min_periods=14).mean()
)
historical_data.loc[:, "Avg_Loss"] = (
    historical_data["Loss"].rolling(window=14, min_periods=14).mean()
)
historical_data.loc[:, "Rs"] = historical_data["Avg_Gain"] / historical_data["Avg_Loss"]

# Crear DataFrames independientes (opcional)
Delta_df = historical_data.loc[:, ["Date", "Delta"]].dropna().reset_index(drop=True)
Gain_df = historical_data.loc[:, ["Date", "Gain"]].dropna().reset_index(drop=True)
Loss_df = historical_data.loc[:, ["Date", "Loss"]].dropna().reset_index(drop=True)
Rs_df = historical_data.loc[:, ["Date", "Rs"]].dropna().reset_index(drop=True)

# Revisa el resultado
historical_data = historical_data.dropna()

historical_data

,Date,Open,High,Low,Close,Adj Close,Volume,SMA_5,SMA_10,SMA_20,...,Log_Return,Middle_Band,Upper_Band,Lower_Band,Delta,Gain,Loss,Avg_Gain,Avg_Loss,Rs
244,2001-01-11,2.962789,2.962789,2.845528,2.845528,2.166392,31410000,2.997005,2.830355,2.870639,...,-0.026266,2.870639,3.255335,2.485942,-0.075731,0.000000,0.075731,0.044544,0.047654,0.934732
245,2001-01-12,2.970607,3.040963,2.947154,3.001876,2.285424,52900000,2.980634,2.876765,2.879374,...,0.053489,2.879374,3.267830,2.490917,0.156348,0.156348,0.000000,0.051223,0.047654,1.074898
247,2001-01-16,3.084593,3.271538,2.998909,3.224801,2.455144,52065000,3.011475,2.946949,2.889431,...,0.071634,2.889431,3.303207,2.475655,0.222925,0.222925,0.000000,0.067147,0.045359,1.480351
248,2001-01-17,3.174355,3.315090,3.174355,3.299453,2.511979,52570000,3.058584,3.014971,2.907469,...,0.022885,2.907469,3.359931,2.455006,0.074651,0.074651,0.000000,0.072479,0.032675,2.218163
249,2001-01-18,3.297045,3.522551,3.297045,3.421462,2.604869,57685000,3.158624,3.092076,2.919998,...,0.036311,2.919998,3.415047,2.424949,0.122009,0.122009,0.000000,0.081194,0.025141,3.229565
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5489,2022-05-17,52.037348,53.053092,52.037348,52.818690,52.818690,15680447,51.695575,52.286403,52.838073,...,0.017635,52.838073,55.318167,50.357980,0.923295,0.923295,0.000000,0.337996,0.374649,0.902167
5490,2022-05-18,53.921336,54.237128,53.368702,53.763441,53.763441,16486319,52.154894,52.332181,52.810646,...,0.017729,52.810646,55.233584,50.387709,0.944751,0.944751,0.000000,0.405479,0.344764,1.176104
5491,2022-05-19,52.133147,52.995500,52.133147,52.917104,52.917104,17073727,52.583013,52.295294,52.732577,...,-0.015867,52.732577,55.026423,50.438732,-0.846336,0.000000,0.846336,0.405479,0.308417,1.314707
5492,2022-05-20,53.658344,54.133196,53.579201,53.816628,53.816628,12109671,53.042252,52.292598,52.704790,...,0.016856,52.704790,54.927353,50.482227,0.899523,0.899523,0.000000,0.469730,0.280937,1.672015


## Paso 2: Entrenamiento de Redes Neuronales

### Preparación de Datos Históricos de Acciones

In [42]:
historical_features = [
    "Open",
    "High",
    "Low",
    "Close",
    "Volume",
    "SMA_5",
    "SMA_10",
    "SMA_20",
    "SMA_50",
    "SMA_100",
    "SMA_200",
    "RSI_14",
    "MACD",
    "Signal_Line",
    "EMA_50",
    "Daily_Return",
    "Log_Return",
    "Middle_Band",
    "Upper_Band",
    "Lower_Band",
]
data_hist = historical_data[historical_features].values

# Normalizar los datos históricos
scaler_hist = MinMaxScaler(feature_range=(0, 1))
data_hist = scaler_hist.fit_transform(data_hist)

# Crear secuencias de tiempo para datos históricos
time_steps = 60
X_hist, y_hist = [], []
for i in range(len(data_hist) - time_steps):
    X_hist.append(data_hist[i : i + time_steps])
    y_hist.append(
        data_hist[i + time_steps, 3]
    )  # Usamos 'Close' como etiqueta (índice 3)
X_hist = np.array(X_hist)
y_hist = np.array(y_hist)


# Dividir los datos en conjuntos de entrenamiento y prueba
train_size_hist = int(len(X_hist) * 0.8)
X_train_hist, X_test_hist = X_hist[:train_size_hist], X_hist[train_size_hist:]
y_train_hist, y_test_hist = y_hist[:train_size_hist], y_hist[train_size_hist:]

# Convertir a tensores de PyTorch
X_train_hist = torch.tensor(X_train_hist, dtype=torch.float32)
y_train_hist = torch.tensor(y_train_hist, dtype=torch.float32)
X_test_hist = torch.tensor(X_test_hist, dtype=torch.float32)
y_test_hist = torch.tensor(y_test_hist, dtype=torch.float32)

### Definición de Modelo LSTM

In [43]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(self.dropout(out[:, -1, :]))
        return out

### Entrenamiento de Histórico de Acciones

In [44]:
# Parámetros del modelo
input_size_hist = len(historical_features)
hidden_size = 50
num_layers = 2
output_size = 1
num_epochs = 50
batch_size = 64
learning_rate = 0.001

# Definir y entrenar el modelo LSTM para datos históricos
model_hist = LSTMModel(input_size_hist, hidden_size, num_layers, output_size)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model_hist.parameters(), lr=learning_rate)

train_loader_hist = DataLoader(
    TensorDataset(X_train_hist, y_train_hist), batch_size=batch_size, shuffle=True
)
test_loader_hist = DataLoader(
    TensorDataset(X_test_hist, y_test_hist), batch_size=batch_size, shuffle=False
)

# Entrenamiento del Modelo
def train_model(model, train_loader, criterion, optimizer, num_epochs):
    for epoch in range(num_epochs):
        model.train()
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            outputs = model(X_batch)
            y_batch = y_batch.view(-1, 1)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

train_model(model_hist, train_loader_hist, criterion, optimizer, num_epochs)

Epoch [1/50], Loss: 0.0003
Epoch [2/50], Loss: 0.0004
Epoch [3/50], Loss: 0.0002
Epoch [4/50], Loss: 0.0003
Epoch [5/50], Loss: 0.0001
Epoch [6/50], Loss: 0.0003
Epoch [7/50], Loss: 0.0002
Epoch [8/50], Loss: 0.0001
Epoch [9/50], Loss: 0.0003
Epoch [10/50], Loss: 0.0002
Epoch [11/50], Loss: 0.0003
Epoch [12/50], Loss: 0.0003
Epoch [13/50], Loss: 0.0002
Epoch [14/50], Loss: 0.0002
Epoch [15/50], Loss: 0.0004
Epoch [16/50], Loss: 0.0001
Epoch [17/50], Loss: 0.0002
Epoch [18/50], Loss: 0.0002
Epoch [19/50], Loss: 0.0001
Epoch [20/50], Loss: 0.0001
Epoch [21/50], Loss: 0.0002
Epoch [22/50], Loss: 0.0001
Epoch [23/50], Loss: 0.0002
Epoch [24/50], Loss: 0.0002
Epoch [25/50], Loss: 0.0001
Epoch [26/50], Loss: 0.0001
Epoch [27/50], Loss: 0.0001
Epoch [28/50], Loss: 0.0001
Epoch [29/50], Loss: 0.0001
Epoch [30/50], Loss: 0.0000
Epoch [31/50], Loss: 0.0001
Epoch [32/50], Loss: 0.0001
Epoch [33/50], Loss: 0.0001
Epoch [34/50], Loss: 0.0000
Epoch [35/50], Loss: 0.0001
Epoch [36/50], Loss: 0.0001
E

### Evaluación de Modelo Histórico

In [45]:
# Evaluación del modelo obtenido:
model_hist.eval()
with torch.no_grad():
    preds = model_hist(X_test_hist).detach().cpu().numpy().flatten()
    targets = y_test_hist.view(-1, 1).detach().cpu().numpy().flatten()
    print("Ejemplo de Predicciones vs. Targets:")
    for p, t in zip(preds[:10], targets[:10]):
        print(f"Predicción: {p:.4f}, Target: {t:.4f}")

Ejemplo de Predicciones vs. Targets:
Predicción: 0.4900, Target: 0.4938
Predicción: 0.5002, Target: 0.5147
Predicción: 0.5143, Target: 0.5308
Predicción: 0.5232, Target: 0.5331
Predicción: 0.5244, Target: 0.5212
Predicción: 0.5149, Target: 0.5168
Predicción: 0.5108, Target: 0.5097
Predicción: 0.5064, Target: 0.5124
Predicción: 0.5073, Target: 0.5167
Predicción: 0.5130, Target: 0.5194


## Desnormalizar Valores

### Función para desnormalizar

In [46]:
def denormalize_value(norm_val, scaler, feature_index, n_features):
    # norm_val: valor normalizado (o un array de valores, uno por muestra)
    # scaler: objeto MinMaxScaler que ya se ajustó con los datos
    # feature_index: el índice de la columna "Close" (en tu caso, 3)
    # n_features: número total de features (len(historical_features))
    # Si norm_val es un array 1D, lo convertimos a 2D
    norm_val = np.array(norm_val).reshape(-1, 1)
    # Creamos un array dummy de ceros con la misma cantidad de filas y n_features columnas
    dummy = np.zeros((norm_val.shape[0], n_features))
    # Asignamos los valores normalizados en la columna 'feature_index'
    dummy[:, feature_index] = norm_val[:, 0]
    # Desnormalizamos usando el inverso del scaler
    inv = scaler.inverse_transform(dummy)
    # Extraemos i.e. "Close"
    return inv[:, feature_index]

In [47]:
n_features_hist = len(historical_features)
baseline_preds_norm = (
    X_test_hist[:, -1, 3].detach().cpu().numpy()
)  # Usamos el "Close" del último timestep
model_hist.eval()
with torch.no_grad():
    model_preds_norm = model_hist(X_test_hist).detach().cpu().numpy().flatten()
    y_test_norm = y_test_hist.detach().cpu().numpy().flatten()
model_preds = denormalize_value(model_preds_norm, scaler_hist, feature_index=3, n_features=n_features_hist)
y_test_actual = denormalize_value(y_test_norm, scaler_hist, feature_index=3, n_features=n_features_hist)
baseline_preds = denormalize_value(baseline_preds_norm, scaler_hist, feature_index=3, n_features=n_features_hist)
mse_model = np.mean((model_preds - y_test_actual) ** 2)
mse_baseline = np.mean((baseline_preds - y_test_actual) ** 2)
print(f"MSE Modelo LSTM (desnormalizado): {mse_model:.4f}")
print(f"MSE Línea Base (desnormalizado): {mse_baseline:.4f}")

MSE Modelo LSTM (desnormalizado): 4.8275
MSE Línea Base (desnormalizado): 0.8274


### Graficar y Visualizar

In [48]:
test_dates = historical_data["Date"].iloc[train_size_hist + time_steps :].values
import plotly.graph_objects as go
test_dates_str = [str(date) for date in test_dates]
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=test_dates_str, y=y_test_actual, mode="lines+markers", name="Real")
)
fig.add_trace(
    go.Scatter(
        x=test_dates_str, y=model_preds, mode="lines+markers", name="Predicción LSTM"
    )
)
fig.add_trace(
    go.Scatter(
        x=test_dates_str, y=baseline_preds, mode="lines+markers", name="Línea Base"
    )
)
fig.update_layout(
    title="Comparación: Modelo LSTM vs. Línea Base vs. Valores Reales",
    xaxis_title="Fecha",
    yaxis_title="Precio de cierre (desnormalizado)",
    template="plotly_white",
)
fig.show()

# Guardamos el Modelo antes de subir a repositorio

In [49]:
import os

directory = r"C:\Users\mleonet_\Desktop\FinalProyect-TeamGalaxy-SIC\Models"
model_filename = "modelo_lstm_historico.pth"
optimizer_filename = "optimizer_lstm_historico.pth"

model_save_path = os.path.join(directory, model_filename)
optimizer_save_path = os.path.join(directory, optimizer_filename)

torch.save(model_hist.state_dict(), model_save_path)
torch.save(optimizer.state_dict(), optimizer_save_path)

print(f"Modelo guardado en {model_save_path}")
print(f"Estado del optimizador guardado en {optimizer_save_path}")

Modelo guardado en C:\Users\mleonet_\Desktop\FinalProyect-TeamGalaxy-SIC\Models\modelo_lstm_historico.pth
Estado del optimizador guardado en C:\Users\mleonet_\Desktop\FinalProyect-TeamGalaxy-SIC\Models\optimizer_lstm_historico.pth
